## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Getting Dataset

In [2]:
dataset = pd.read_csv("Data.csv")

In [12]:
dataset

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes
5,France,35.0,58000.0,Yes
6,Spain,NaN,52000.0,No
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
Country      10 non-null object
Age          9 non-null float64
Salary       9 non-null float64
Purchased    10 non-null object
dtypes: float64(2), object(2)
memory usage: 448.0+ bytes


In [6]:
dataset.describe()

,Age,Salary
count,9.000000,9.000000
mean,38.777778,63777.777778
std,7.693793,12265.579662
min,27.000000,48000.000000
25%,35.000000,54000.000000
50%,38.000000,61000.000000
75%,44.000000,72000.000000
max,50.000000,83000.000000


## Finding missing data

In [11]:
dataset[dataset.Salary.isna()==True]

,Country,Age,Salary,Purchased
4,Germany,40.0,NaN,Yes


In [13]:
dataset[dataset.Age.isna()==True]

,Country,Age,Salary,Purchased
6,Spain,NaN,52000.0,No


Splitting dependent and independent variables:

In [16]:
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

In [27]:
qualitative = [column for column in X.columns if X[column].dtype=="object"]
quantitative = [column for column in X.columns if column not in qualitative]

In [30]:
print(qualitative)
print(quantitative)

['Country']
['Age', 'Salary']


### 1. Imputing missing data

In [14]:
from sklearn.preprocessing import Imputer

In [15]:
missing_imputer = Imputer(strategy="mean")

In [31]:
X[quantitative] = missing_imputer.fit_transform(X[quantitative])

In [32]:
X

,Country,Age,Salary
0,France,44.000000,72000.000000
1,Spain,27.000000,48000.000000
2,Germany,30.000000,54000.000000
3,Spain,38.000000,61000.000000
4,Germany,40.000000,63777.777778
5,France,35.000000,58000.000000
6,Spain,38.777778,52000.000000
7,France,48.000000,79000.000000
8,Germany,50.000000,83000.000000
9,France,37.000000,67000.000000


In [33]:
dataset.Salary.mean()

63777.77777777778

In [34]:
dataset.Age.mean()

38.77777777777778

### 2. Enconding categorical data

In [35]:
from sklearn.preprocessing import OneHotEncoder

In [87]:
ohe = OneHotEncoder()
categorical = ohe.fit_transform(X[qualitative])
categorical_df = pd.DataFrame(data=categorical.toarray(), columns=ohe.categories_)

In [90]:
X_final = pd.merge(categorical_df, X[quantitative], how="inner", left_index=True, right_index=True)

In [91]:
X_final

,"(France,)","(Germany,)","(Spain,)",Age,Salary
0,1.0,0.0,0.0,44.000000,72000.000000
1,0.0,0.0,1.0,27.000000,48000.000000
2,0.0,1.0,0.0,30.000000,54000.000000
3,0.0,0.0,1.0,38.000000,61000.000000
4,0.0,1.0,0.0,40.000000,63777.777778
5,1.0,0.0,0.0,35.000000,58000.000000
6,0.0,0.0,1.0,38.777778,52000.000000
7,1.0,0.0,0.0,48.000000,79000.000000
8,0.0,1.0,0.0,50.000000,83000.000000
9,1.0,0.0,0.0,37.000000,67000.000000


In [92]:
from sklearn.preprocessing import LabelBinarizer

In [100]:
lb = LabelBinarizer()
y_final = lb.fit_transform(y)

In [101]:
y_final

array([[0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1]])

### 3. Pipeline

In [170]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, Imputer

In [171]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

In [172]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, data, labels=None):
        return self

    def transform(self, data):
        assert isinstance(data, pd.DataFrame)

        try:
            return data[self.columns]
        except KeyError:
            cols_error = list(set(self.columns) - set(data.columns))
            raise KeyError("The DataFrame does not include the columns: %s" % cols_error)

In [173]:
numerical = Pipeline(steps=[
    ("numerical_columns", ColumnSelector(columns=quantitative)),
    ("mean_imputer", Imputer(strategy="mean"))
            ])

categorical = Pipeline(steps=[
    ("categorical_columns", ColumnSelector(columns=qualitative)),
    ("onehot", OneHotEncoder())
            ])

preprocessor = FeatureUnion(transformer_list=[
            ('cat', categorical),
            ('num', numerical)
                ])

In [174]:
X_final = preprocessor.fit_transform(X)

In [175]:
X_final = pd.DataFrame(X_final.toarray(), columns=list(categorical[1].categories_[0])+quantitative)

In [176]:
X_final

,France,Germany,Spain,Age,Salary
0,1.0,0.0,0.0,44.000000,72000.000000
1,0.0,0.0,1.0,27.000000,48000.000000
2,0.0,1.0,0.0,30.000000,54000.000000
3,0.0,0.0,1.0,38.000000,61000.000000
4,0.0,1.0,0.0,40.000000,63777.777778
5,1.0,0.0,0.0,35.000000,58000.000000
6,0.0,0.0,1.0,38.777778,52000.000000
7,1.0,0.0,0.0,48.000000,79000.000000
8,0.0,1.0,0.0,50.000000,83000.000000
9,1.0,0.0,0.0,37.000000,67000.000000


In [154]:
numerical = Pipeline(steps=[
    ("numerical_columns", ColumnSelector(columns=quantitative)),
    ("mean_imputer", Imputer(strategy="mean")),
    ("std_scaler", StandardScaler())
            ])

categorical = Pipeline(steps=[
    ("categorical_columns", ColumnSelector(columns=qualitative)),
    ("onehot", OneHotEncoder())
            ])

preprocessor = FeatureUnion(transformer_list=[
            ('cat', categorical),
            ('num', numerical)
                ])

In [155]:
X_final = preprocessor.fit_transform(X)

In [156]:
X_final = pd.DataFrame(X_final.toarray(), columns=list(categorical[1].categories_[0])+quantitative)

In [157]:
X_final

,France,Germany,Spain,Age,Salary
0,1.0,0.0,0.0,0.758874,7.494733e-01
1,0.0,0.0,1.0,-1.711504,-1.438178e+00
2,0.0,1.0,0.0,-1.275555,-8.912655e-01
3,0.0,0.0,1.0,-0.113024,-2.532004e-01
4,0.0,1.0,0.0,0.177609,6.632192e-16
5,1.0,0.0,0.0,-0.548973,-5.266569e-01
6,0.0,0.0,1.0,0.000000,-1.073570e+00
7,1.0,0.0,0.0,1.340140,1.387538e+00
8,0.0,1.0,0.0,1.630773,1.752147e+00
9,1.0,0.0,0.0,-0.258340,2.937125e-01


## Splitting dataset into training and testing

In [190]:
from sklearn.model_selection import train_test_split

In [191]:
X_train, X_test, y_train, y_test = train_test_split(X, y_final, test_size=0.2, random_state=0)

Applying pipeline:

In [192]:
X_train_final = preprocessor.fit_transform(X_train)
X_test_final = preprocessor.fit(X_test)